In [170]:
import brainlit
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
from brainlit.algorithms.generate_fragments import adaptive_thresh, tube_seg

In [171]:
data_dir = "benchmarking_datasets/"

In [172]:
from skimage import io

In [173]:
im = io.imread("/Users/mattf/Documents/2020_Courses/NDD/benchmarking_datasets/validation_1-gfp.tif")

In [174]:
!pwd

/Users/mattf/Documents/2020_Courses/NDD/mouselit/matt


In [175]:
# Reference: https://github.com/NeuroDataDesign/mouselit/blob/master/bijan/mouse_test/final%20notebook.ipynb

import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
import napari


def plot_2d(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayFromImage(img)
    spacing = img.GetSpacing()

    if nda.ndim == 3:
        c = nda.shape[-1]

        if c not in (3, 4):
            nda = nda[nda.shape[0] // 2, :, :]

    elif nda.ndim == 4:
        c = nda.shape[-1]

        if c not in (3, 4):
            raise RuntimeError("Unable to show 3D-vector Image")

        nda = nda[nda.shape[0] // 2, :, :, :]

    xsize = nda.shape[1] * 2
    ysize = nda.shape[0] * 2

    figsize = (1 + margin) * xsize / dpi, (1 + margin) * ysize / dpi

    plt.figure(figsize=figsize, dpi=dpi, tight_layout=True)
    ax = plt.gca()

    extent = (0, xsize * spacing[0], ysize * spacing[1], 0)

    t = ax.imshow(nda, extent=extent, interpolation=None)

    if nda.ndim == 2:
        t.set_cmap("gray")

    if title:
        plt.title(title)

    plt.show()


def plot_3d(img, xslices=[], yslices=[], zslices=[], title=None, margin=0.05, dpi=80):
    img_xslices = [img[s, :, :] for s in xslices]
    img_yslices = [img[:, s, :] for s in yslices]
    img_zslices = [img[:, :, s] for s in zslices]

    maxlen = max(len(img_xslices), len(img_yslices), len(img_zslices))

    img_null = sitk.Image([0, 0], img.GetPixelID(), img.GetNumberOfComponentsPerPixel())

    img_slices = []
    d = 0

    if len(img_xslices):
        img_slices += img_xslices + [img_null] * (maxlen - len(img_xslices))
        d += 1

    if len(img_yslices):
        img_slices += img_yslices + [img_null] * (maxlen - len(img_yslices))
        d += 1

    if len(img_zslices):
        img_slices += img_zslices + [img_null] * (maxlen - len(img_zslices))
        d += 1

    if maxlen != 0:
        if img.GetNumberOfComponentsPerPixel() == 1:
            img = sitk.Tile(img_slices, [maxlen, d])
        else:
            img_comps = []
            for i in range(0, img.GetNumberOfComponentsPerPixel()):
                img_slices_c = [sitk.VectorIndexSelectionCast(s, i) for s in img_slices]
                img_comps.append(sitk.Tile(img_slices_c, [maxlen, d]))
            img = sitk.Compose(img_comps)

    plot_2d(img, title, margin, dpi)


def napari_viewer(img, labels=None, shapes=None, label_name="Segmentation"):
    #with napari.gui_qt():
    viewer = napari.view_image(np.squeeze(np.array(img)))
    if labels is not None:
        viewer.add_labels(labels, name=label_name)
    if shapes is not None:
        viewer.add_shapes(data=shapes, shape_type='path', edge_color='blue', name='Skeleton')
    return viewer

In [176]:
import numpy as np

In [177]:
%gui qt
napari_viewer(im)

In [178]:
im.shape

(100, 330, 330)

In [179]:
import pandas as pd

In [180]:
swc_path = "/Users/mattf/Documents/2020_Courses/NDD/benchmarking_datasets/Manual-GT/10-01_validation_1-5/10-01_validation_1/tree_1.swc"

In [181]:
df = pd.read_csv(swc_path, skiprows = 3, header = None, sep = " ")

In [182]:
subneuron_df = df

In [183]:
subneuron_df

,0,1,2,3,4,5,6
0,1,0,13.946604,-30.383750,-13.340353,1.0,-1
1,2,0,13.946604,-30.082379,-12.289681,1.0,1
2,3,0,13.647943,-29.781007,-11.239009,1.0,2
3,4,0,13.647943,-29.479635,-11.239009,1.0,3
4,5,0,11.855971,-27.671405,-11.239009,1.0,4
...,...,...,...,...,...,...,...
122,123,0,-9.946345,26.876879,11.875780,1.0,122
123,124,0,-9.946345,27.479623,11.875780,1.0,123
124,125,0,-9.946345,27.780995,11.875780,1.0,124
125,126,0,-10.842331,28.685110,11.875780,1.0,125


In [184]:
subneuron_df.columns = ["sample",  "structure",      "x",      "y",      "z",
                         "r",  "parent"]

In [185]:
G = df_to_graph(subneuron_df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
G.nodes[1]

Number of nodes: 127
Number of edges: 126


Sample 1 coordinates (x,y,z)


{'x': 13, 'y': -30, 'z': -13}

In [186]:
paths = graph_to_paths(G)
len(paths)

1

In [202]:
paths2 = paths[0] + offset_diff

In [223]:
(paths[0] + offset_diff)/(scales)*1000

array([[240.62319183, -19.57104823,  28.87629221],
       [240.62319183, -19.57104823,  29.88802411],
       [240.62319183, -16.28606868,  30.899756  ],
       [240.62319183, -16.28606868,  30.899756  ],
       [233.92883805,  -9.71610959,  30.899756  ],
       [233.92883805,  -9.71610959,  30.899756  ],
       [230.58166116,  -3.1461505 ,  30.899756  ],
       [230.58166116,  -3.1461505 ,  31.91148789],
       [227.23448427,  -3.1461505 ,  31.91148789],
       [227.23448427,   3.4238086 ,  31.91148789],
       [227.23448427,   3.4238086 ,  31.91148789],
       [227.23448427,   3.4238086 ,  31.91148789],
       [223.88730738,   3.4238086 ,  31.91148789],
       [223.88730738,   6.70878814,  31.91148789],
       [223.88730738,   9.99376769,  31.91148789],
       [223.88730738,   9.99376769,  31.91148789],
       [220.54013049,   9.99376769,  31.91148789],
       [220.54013049,  13.27874724,  31.91148789],
       [220.54013049,  19.84870633,  31.91148789],
       [220.54013049,  23.13368

In [224]:
paths/(scales)*1000

array([[[ 43.51329956, -98.54938639, -13.15251463],
        [ 43.51329956, -98.54938639, -12.14078274],
        [ 43.51329956, -95.26440685, -11.12905084],
        [ 43.51329956, -95.26440685, -11.12905084],
        [ 36.81894578, -88.69444776, -11.12905084],
        [ 36.81894578, -88.69444776, -11.12905084],
        [ 33.47176889, -82.12448866, -11.12905084],
        [ 33.47176889, -82.12448866, -10.11731895],
        [ 30.124592  , -82.12448866, -10.11731895],
        [ 30.124592  , -75.55452957, -10.11731895],
        [ 30.124592  , -75.55452957, -10.11731895],
        [ 30.124592  , -75.55452957, -10.11731895],
        [ 26.77741511, -75.55452957, -10.11731895],
        [ 26.77741511, -72.26955002, -10.11731895],
        [ 26.77741511, -68.98457048, -10.11731895],
        [ 26.77741511, -68.98457048, -10.11731895],
        [ 23.43023823, -68.98457048, -10.11731895],
        [ 23.43023823, -65.69959093, -10.11731895],
        [ 23.43023823, -59.12963184, -10.11731895],
        [ 23

In [225]:
%gui qt
napari_viewer(im.T, shapes=((paths[0] + offset_diff)/(scales)*1000))

In [ ]:
(sample + offset_diff)*1000/scales

In [191]:
scales = np.array([298.75923295, 304.41589844, 988.40414663])
swc_offset = [73448.511531, 14630.940220, 33145.238458]
brain_offset = [69445.19581378, 12917.40798423, 30199.63896704]
vol_offset = [3944.427317, 1689.489974, 2904.058044]

In [192]:
im_offset = np.add(brain_offset, vol_offset)
print(im_offset)

[73389.62313078 14606.89795823 33103.69701104]


In [193]:
offset_diff = np.subtract(swc_offset, im_offset)
print(offset_diff)

[58.88840022 24.04226177 41.54144696]


In [194]:
im.shape

(100, 330, 330)

In [195]:
coord = np.array([13.946604, -30.383750, -13.340353])

In [196]:
offset_diff

array([58.88840022, 24.04226177, 41.54144696])

In [197]:
point = coord + offset_diff
print(point)

[72.83500422 -6.34148823 28.20109396]


In [163]:
np.divide(point, scales)

array([ 0.24379164, -0.02083166,  0.02853195])

In [164]:
np.divide(point, scales*1000)

array([ 2.43791643e-04, -2.08316591e-05,  2.85319462e-05])

In [165]:
np.divide(point, scales/1000)

array([243.79164286, -20.83165913,  28.53194622])

In [83]:
print(np.divide(offset,scales))
o = np.divide(offset,scales)

[247.54835715  57.46040476  34.76714286]


In [84]:
paths[0] = paths[0] + o

In [89]:
skel_path = "/Users/mattf/Documents/2020_Courses/NDD/benchmarking_datasets/Manual-GT/8-01_test_1-5/8-01_test_1/tree_1.swc"
skel_graph = 

array([[[11968, 11970, 11837, ..., 11760, 11698, 11787],
        [11817, 11795, 11764, ..., 11748, 11732, 11758],
        [11750, 11813, 11865, ..., 11907, 11792, 11734],
        ...,
        [12061, 12096, 11861, ..., 11824, 11817, 11859],
        [11884, 11868, 11947, ..., 11896, 11804, 11716],
        [11912, 11846, 11996, ..., 11774, 11714, 11707]],

       [[11834, 11839, 11889, ..., 11778, 11803, 11887],
        [11810, 11824, 11817, ..., 11750, 11797, 11799],
        [11886, 11865, 11698, ..., 11788, 11878, 11820],
        ...,
        [11864, 11859, 11898, ..., 11847, 11852, 11897],
        [11703, 11737, 11878, ..., 11868, 11839, 11812],
        [11797, 11926, 12083, ..., 11869, 11895, 11807]],

       [[11782, 11648, 11768, ..., 11879, 11879, 11794],
        [11756, 11714, 11691, ..., 11872, 11817, 11752],
        [11869, 11818, 11725, ..., 11837, 11758, 11714],
        ...,
        [11867, 11936, 11834, ..., 11787, 11829, 11927],
        [11871, 11791, 11820, ..., 11775, 117

In [104]:
%gui qt
napari_viewer(im.swapaxes(0,1).swapaxes(1,2), shapes=paths)

In [102]:
print(im.shape)
print(im.T.shape)
print(im.swapaxes(0,1).swapaxes(1,2).shape)

(100, 330, 330)
(330, 330, 100)
(330, 330, 100)


In [216]:
print(offset_diff)

[58.88840022 24.04226177 41.54144696]


In [226]:
#scales = [298.75923295, 304.41589844, 988.40414663]
#offset = [73957.357299, 17491.860740, 34363.988167]
seed = []
for index, row in subneuron_df.iterrows():
    sample = np.array((row['x'], row['y'], row['z']))
    #voxel = sample + np.divide(offset, scales)
    #print(sample+ offset_diff)
    voxel = (sample + offset_diff)/(scales)*1000
    #print(sample, voxel)
    #voxel = np.divide(sample + offset_diff, scales)
    #voxel = (np.divide(sample-offset, scales)) #).astype(int)
    #print(sample, voxel)
    _, s = adaptive_thresh.get_seed(voxel)
    seed.append(s)
print(seed)

[(28, -20, 243), (29, -19, 243), (30, -18, 242), (30, -17, 242), (30, -11, 236), (30, -10, 236), (30, -5, 231), (31, -4, 230), (31, -4, 229), (31, 0, 229), (31, 0, 228), (31, 1, 227), (31, 2, 226), (31, 4, 226), (31, 6, 224), (31, 7, 224), (31, 9, 222), (31, 11, 222), (31, 19, 222), (32, 20, 222), (32, 21, 222), (32, 22, 221), (32, 23, 221), (32, 24, 220), (32, 25, 220), (32, 26, 219), (32, 28, 219), (33, 29, 218), (33, 31, 216), (33, 32, 216), (34, 33, 215), (34, 34, 215), (34, 37, 212), (35, 38, 212), (35, 39, 213), (35, 40, 213), (35, 41, 212), (35, 40, 213), (38, 42, 211), (38, 45, 211), (38, 47, 209), (38, 48, 208), (38, 48, 206), (38, 49, 205), (39, 50, 204), (39, 51, 204), (39, 53, 202), (40, 53, 202), (41, 54, 202), (41, 55, 203), (41, 57, 203), (42, 58, 203), (42, 60, 203), (42, 62, 201), (42, 63, 201), (42, 64, 200), (42, 65, 200), (42, 70, 195), (42, 72, 195), (42, 73, 194), (42, 83, 194), (42, 84, 194), (43, 85, 193), (43, 86, 193), (43, 87, 192), (43, 89, 192), (43, 90, 19

In [2]:
seed.shape

NameError: name 'seed' is not defined

In [169]:
%gui qt
napari_viewer(im, shapes=paths)

In [18]:
seed = []
for index, row in subneuron_df.iterrows():
    sample = (row['x'], row['y'], row['z'])
    _, s = adaptive_thresh.get_seed(sample)
    seed.append(s)
print(seed)

[(-53, -61, -61), (46, -61, -61), (46, 38, -61), (46, -61, 38), (46, 38, 38), (-53, -61, 38), (-53, 38, 38), (-53, 38, -61), (-53, 38, 38), (46, 38, 38), (46, 38, -61), (46, 38, 38), (-53, -61, 38)]


In [19]:
print(im.shape)

(100, 330, 330)


In [42]:
true = np.array([-61.406513, -61.445178, -53.753979])
offset = np.array([73963.685414, 17492.299523, 34367.192334])
scales = [
    [298.75923295, 304.41589844, 988.40414663], [304.41589844, 988.40414663, 298.75923295], [298.75923295, 304.41589844, 988.40414663],
    [298759.23295, 304415.89844, 988404.14663], [304415.89844, 988404.14663, 298759.23295], [298759.23295, 304415.89844, 988404.14663],
    [.29875923295, .30441589844, .98840414663]
]
for scale in scales:
    print((np.divide(true+offset, scale)))

[247.36400001  57.26        34.716     ]
[242.76747463  17.63535129 114.85314785]
[247.36400001  57.26        34.716     ]
[0.247364 0.05726  0.034716]
[0.24276747 0.01763535 0.11485315]
[0.247364 0.05726  0.034716]
[247364.00000521  57260.00000107  34716.0000006 ]


In [45]:
true = np.array([38.379071 -61.445178 -53.753979])
offset = np.array([73963.685414, 17492.299523, 34367.192334])
scales = [
    [298.75923295, 304.41589844, 988.40414663], [304.41589844, 988.40414663, 298.75923295], [298.75923295, 304.41589844, 988.40414663],
    [298759.23295, 304415.89844, 988404.14663], [304415.89844, 988404.14663, 298759.23295], [298759.23295, 304415.89844, 988404.14663],
    [.29875923295, .30441589844, .98840414663]
]
for scale in scales:
    print((np.divide(true+offset, scale)))

[247.31240805  57.20949374  34.69266328]
[242.71684136  17.61979601 114.77594151]
[247.31240805  57.20949374  34.69266328]
[0.24731241 0.05720949 0.03469266]
[0.24271684 0.0176198  0.11477594]
[0.24731241 0.05720949 0.03469266]
[247312.40804988  57209.49374276  34692.66328446]


In [51]:
true = np.array([13048, 8009, 5113])
offset = np.array([73963.685414, 17492.299523, 34367.192334])
scales = [
    [298.75923295, 304.41589844, 988.40414663], [304.41589844, 988.40414663, 298.75923295], [298.75923295, 304.41589844, 988.40414663],
    [298759.23295, 304415.89844, 988404.14663], [304415.89844, 988404.14663, 298759.23295], [298759.23295, 304415.89844, 988404.14663],
    [.29875923295, .30441589844, .98840414663]
]
for scale in scales:
    print((np.divide(true+offset, scale)))

[291.24350252  83.77124734  39.94336979]
[285.83160689  25.80047808 132.14718737]
[291.24350252  83.77124734  39.94336979]
[0.2912435  0.08377125 0.03994337]
[0.28583161 0.02580048 0.13214719]
[0.2912435  0.08377125 0.03994337]
[291243.50251817  83771.24734182  39943.36979323]


In [32]:
scale = np.divide([25600, 13312, 33792], 1000)
print((np.divide(sample-offset, scale)))

[-2887.70727902 -1318.64067766 -1018.61228436]


KeyError: 1

In [24]:
scales = np.divide([298.75923295, 304.41589844, 988.40414663],[1000,1000,1000])
offset = [73963.685414, 17492.299523, 34367.192334]
seed = []
for index, row in subneuron_df.iterrows():
    sample = np.array((row['x'], row['y'], row['z']))
    voxel = np.round((np.divide(sample-offset, scales))).astype(int)
    print(sample, voxel)
    _, s = adaptive_thresh.get_seed(sample)
    seed.append(s)
#print(seed)

[-61.406513 -61.445178 -53.753979] [-247775  -57664  -34825]
[-61.406513 -61.445178  46.074839] [-247775  -57664  -34724]
[-61.406513  38.403236  46.074839] [-247775  -57336  -34724]
[ 38.379071 -61.445178  46.074839] [-247441  -57664  -34724]
[38.379071 38.403236 46.074839] [-247441  -57336  -34724]
[ 38.379071 -61.445178 -53.753979] [-247441  -57664  -34825]
[ 38.379071  38.403236 -53.753979] [-247441  -57336  -34825]
[-61.406513  38.403236 -53.753979] [-247775  -57336  -34825]
[ 38.379071  38.403236 -53.753979] [-247441  -57336  -34825]
[38.379071 38.403236 46.074839] [-247441  -57336  -34724]
[-61.406513  38.403236  46.074839] [-247775  -57336  -34724]
[38.379071 38.403236 46.074839] [-247441  -57336  -34724]
[ 38.379071 -61.445178 -53.753979] [-247441  -57664  -34825]
[(-53, -61, -61), (46, -61, -61), (46, 38, -61), (46, -61, 38), (46, 38, 38), (-53, -61, 38), (-53, 38, 38), (-53, 38, -61), (-53, 38, 38), (46, 38, 38), (46, 38, -61), (46, 38, 38), (-53, -61, 38)]


In [1]:
labels = adaptive_thresh.confidence_connected_threshold(im.T, seed, num_iter=1, multiplier=0.3)

NameError: name 'adaptive_thresh' is not defined

In [22]:
## Most useful visualization cell - contains image, SWC, and automatic segmentation
#%gui qt
napari_viewer(img, labels=labels, shapes=paths, label_name="Confidence-Connected Threshold")

NameError: name 'img' is not defined

In [23]:
labels = tube_seg.tubes_seg(img, vox_in_img_list, 2)

NameError: name 'img' is not defined

In [ ]:
## Most useful visualization cell - contains image, SWC, and automatic segmentation
#%gui qt
napari_viewer(img, labels=labels, shapes=paths, label_name="Tube Segmentation")